# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
cities_df = pd.read_csv('output_data/cities.csv', dtype ='object', encoding='utf-8')

cities_df.head()

print(len(cities_df))

cities_df = cities_df.dropna()

cities_df.head()

# change Lat, Lng, Max Temp, Humidity, Wind Speed datatype from string to float
convert_dict = {'Lng': float,
                'Lat': float,
                'Max Temp': float,
                'Humidity': float,
                'Wind Speed': float}
cities_df = cities_df.astype(convert_dict)



535


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)
locations = cities_df[['Lat','Lng']]

humidity = cities_df['Humidity']

In [5]:
fig = gmaps.figure()

humidity_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=2)

fig.add_layer(humidity_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
hotel_df = cities_df.loc[(cities_df['Max Temp'] < 80) & (cities_df['Max Temp'] >= 70) & 
          (cities_df['Wind Speed'] < 10)]

hotel_df.head()

#print(len(hotel_df))


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bengkulu,-3.8004,102.2655,76.10,89.0,100,2.89,ID,1666554984
6,6,porto novo,6.4965,2.6036,78.62,88.0,68,5.44,BJ,1666555163
7,7,kokopo,-4.3520,152.2633,76.93,83.0,100,2.59,PG,1666555163
16,16,voh,-20.9667,164.7000,72.03,90.0,100,6.02,NC,1666555169
20,20,puerto ayora,-0.7393,-90.3518,75.16,86.0,54,5.99,EC,1666555171


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [28]:
hotel_df['Hotel Name'] = ''
top_10_hotels_by_humidity_maxtemp = hotel_df.sort_values(by = ['Humidity','Max Temp'], ascending=True).head(12)
top_10_hotels_by_humidity_maxtemp

/var/folders/jc/lj3w38pd0l1bz1zn9v72dx8h0000gn/T/ipykernel_7377/1243786109.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ''


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
126,126,gwembe,-16.4975,27.6069,74.48,31.0,100,2.26,ZM,1666555235,
351,351,santa maria,-29.6842,-53.8069,76.30,31.0,20,5.75,BR,1666555063,
24,24,jamestown,42.0970,-79.2353,71.24,32.0,0,4.61,US,1666554941,
147,147,kaduna,10.5222,7.4383,75.76,35.0,0,6.55,NG,1666555166,
413,413,santa fe,-31.6333,-60.7000,77.02,36.0,0,7.00,AR,1666555150,
226,226,lundazi,-12.2929,33.1782,75.78,38.0,44,8.50,ZM,1666555299,
333,333,avera,33.1940,-82.5271,77.56,38.0,13,4.54,US,1666555352,
464,464,hamilton,39.1834,-84.5333,79.00,40.0,0,4.00,US,1666555252,
133,133,lompoc,34.6391,-120.4579,75.69,41.0,0,7.00,US,1666555242,
502,502,dentsville,34.0640,-80.9582,75.88,41.0,0,5.75,US,1666555424,


In [29]:
for index, row in top_10_hotels_by_humidity_maxtemp.iterrows():
    target_coordinates = str(top_10_hotels_by_humidity_maxtemp.loc[index,'Lat']) + ',' + str(top_10_hotels_by_humidity_maxtemp.loc[index,'Lng'])
    target_type = 'lodging'
    
    params = {
        'radius':'5000',
        'location':target_coordinates,
        'type':target_type,
        'key':g_key
    }
    
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    try:
        first_result = results[0]
        top_10_hotels_by_humidity_maxtemp.loc[index,'Hotel Name']=first_result.get('name')        
    except:
        print(f'No hotels found for city {str(index)}')
        

    

No hotels found for city 126
No hotels found for city 333


In [35]:
# Remove records with missing hotels
top_10_hotels_by_humidity_maxtemp['Hotel Name'].replace('',np.nan,inplace=True)  
top_10_hotels_by_humidity_maxtemp = top_10_hotels_by_humidity_maxtemp.dropna()

# Print Top hotels
top_10_hotels_by_humidity_maxtemp
hotel_df = top_10_hotels_by_humidity_maxtemp
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
351,351,santa maria,-29.6842,-53.8069,76.30,31.0,20,5.75,BR,1666555063,Hotel Morotin
24,24,jamestown,42.0970,-79.2353,71.24,32.0,0,4.61,US,1666554941,Hampton Inn & Suites Jamestown
147,147,kaduna,10.5222,7.4383,75.76,35.0,0,6.55,NG,1666555166,HAMDALA HOTEL
413,413,santa fe,-31.6333,-60.7000,77.02,36.0,0,7.00,AR,1666555150,Conquistador Hotel
226,226,lundazi,-12.2929,33.1782,75.78,38.0,44,8.50,ZM,1666555299,Knox Lodge
464,464,hamilton,39.1834,-84.5333,79.00,40.0,0,4.00,US,1666555252,Six Acres Bed & Breakfast
133,133,lompoc,34.6391,-120.4579,75.69,41.0,0,7.00,US,1666555242,"Holiday Inn Express Lompoc, an IHG Hotel"
502,502,dentsville,34.0640,-80.9582,75.88,41.0,0,5.75,US,1666555424,Columbia Northeast Hotel
466,466,evans,33.5337,-82.1307,78.04,43.0,0,0.00,US,1666555410,Evans hide out
419,419,kontagora,10.3999,5.4695,78.40,44.0,0,2.48,NG,1666555387,Queens Guest Inn


In [36]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [37]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))